In [1]:
import tensorflow.keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization, LocallyConnected2D, SeparableConv2D
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.constraints import max_norm
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import activations

In [2]:
from import_data import import_data

In [3]:
X, y, _ = import_data()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

153 parties rejetées par le goban, reste 5835 parties
(37344, 9, 9, 4)
(37344, 82)


In [4]:
l2 = 0.01
model = Sequential()
model.add(Conv2D(128, kernel_size=3, input_shape=X[0].shape, padding ="same", kernel_regularizer=regularizers.l2(l2)))
model.add(Conv2D(64, kernel_size=3, padding ="same", kernel_regularizer=regularizers.l2(l2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, padding ="same", kernel_regularizer=regularizers.l2(l2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(l2)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(l2)))
model.add(Dropout(0.2))
model.add(Dense(82, activation='sigmoid', kernel_regularizer=regularizers.l2(l2))) 

#opt = optimizers.RMSprop(learning_rate=0.01, momentum=0.9)
opt = optimizers.Adam()
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_absolute_error'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 128)         4736      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          73792     
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2

In [5]:
# hyperparamètres
epochs = 10
batch_size = 128

# training
print("Training ...")
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Training ...
Epoch 1/10
292/292 [==============================] - 4s 15ms/step - loss: 0.9528 - mean_absolute_error: 0.2074 - val_loss: 0.1498 - val_mean_absolute_error: 0.3251
Epoch 2/10
292/292 [==============================] - 4s 14ms/step - loss: 0.0809 - mean_absolute_error: 0.1995 - val_loss: 0.1049 - val_mean_absolute_error: 0.2668
Epoch 3/10
292/292 [==============================] - 4s 14ms/step - loss: 0.0797 - mean_absolute_error: 0.1982 - val_loss: 0.0920 - val_mean_absolute_error: 0.1733
Epoch 4/10
292/292 [==============================] - 4s 14ms/step - loss: 0.0793 - mean_absolute_error: 0.1973 - val_loss: 0.0826 - val_mean_absolute_error: 0.2214
Epoch 5/10
292/292 [==============================] - 4s 14ms/step - loss: 0.0786 - mean_absolute_error: 0.1967 - val_loss: 0.1437 - val_mean_absolute_error: 0.2286
Epoch 6/10
292/292 [==============================] - 4s 14ms/step - loss: 0.0779 - mean_absolute_error: 0.1960 - val_loss: 0.0788 - val_mean_absolute_error: 0.20

In [31]:
proba = model.predict(X_train)

In [32]:
print(proba[5])

[0.5011088  0.5065884  0.50180507 0.50163645 0.5018956  0.50163513
 0.5015248  0.49934635 0.5011458  0.5031197  0.5042301  0.50360703
 0.50094414 0.49957764 0.50129557 0.5030304  0.50164336 0.5032757
 0.50083464 0.50298893 0.5019576  0.5072333  0.4980696  0.5038169
 0.5007657  0.5045776  0.5009092  0.5007753  0.5002029  0.5054479
 0.5032246  0.4959061  0.5030258  0.50773364 0.5017242  0.50663143
 0.5019292  0.5009095  0.49783373 0.5008707  0.5050464  0.49938262
 0.4964479  0.50281775 0.50165486 0.50129116 0.499834   0.50720555
 0.5037054  0.49855343 0.50552213 0.506981   0.5002532  0.50160986
 0.4979841  0.5006192  0.50021255 0.50713956 0.4958895  0.5070673
 0.5002522  0.50346935 0.50137794 0.5017547  0.50443196 0.5042616
 0.5018245  0.50204754 0.5042344  0.50246674 0.50479895 0.50161195
 0.5024987  0.5001665  0.5016533  0.50163794 0.50364214 0.50026035
 0.500437   0.50349444 0.5012306  0.50256395]
